In [115]:
!pip install --quiet google-cloud-documentai
!pip install --quiet vertexAI
!pip install --quiet --upgrade google-cloud-storage
!pip install --quiet pyPdf2
!pip install --quiet --upgrade google-cloud-aiplatform

In [55]:
document_bucket_uri = "gs://jingle-project-414801/thai-form-scanner/input/"

document_bucket_name = "thai-form-scanner"

download_folder = "forms"

In [3]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

In [172]:
import vertexai
import json
from vertexai.generative_models import GenerativeModel, Part
import typing_extensions as typing
from google.cloud import aiplatform
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

# TODO(developer): Update project_id and location
vertexai.init(project=PROJECT_ID, location="us-central1")

get_form_details = FunctionDeclaration(
    name="get_form_details",
    parameters={
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
            },
            "date of birth": {
                "type": "string",
            },
            "phone number": {
                "type": "string",
            },
            "emergency contact": {
                "type": "string",
                "description":"The name of the person to contact during an emergency"
            },
            "emergency number": {
                "type": "string",
                "description": "The phone number of the emergency contact"
            },
        },
    },
)

form_tool = Tool(
    function_declarations=[get_form_details],
)
   

model = GenerativeModel("gemini-1.5-flash-002",generation_config = GenerationConfig(
        temperature=0
    ))



    

form_prompt = """
You are a professional form parsing expert. You are able to accurately scan through a form and extract the user's
inputs. Take note the fields present in the form, such as Name, Phone Number etc.
Here is an example of one such form:
"""

prompt = """
If the text is in Thai language, you will translate it into English text. You will then return
this data as a JSON object. 

Process the following forms using strictly the data you extract from them:
"""


In [135]:
# Function to create Document AI Processor
def create_processor(project_id, location, processor_display_name, processor_type):
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    processor = client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_=processor_type
        ),
    )
    
    processor_id = processor.name.split('/')[-1]

    # Print the processor information
    print(f"Processor Name: {processor.name}")
    print(f"Processor Display Name: {processor.display_name}")
    print(f"Processor ID: {processor_id}")
    print(f"Processor Type: {processor.type_}")
    
    
    return processor, processor_id

#Function to retrieve list of existing processors
def list_processors(project_id: str, location: str) -> None:
    processorList=[]
    
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(PROJECT_ID, location)

    # Make ListProcessors request
    processor_list = client.list_processors(parent=parent)

    # Print the processor information
    for processor in processor_list:
        # print(f"Processor Name: {processor.name}")
        # print(f"Processor Display Name: {processor.display_name}")
        # print(f"Processor Type: {processor.type_}")
        processorList.append(processor)
        
    return processorList

In [6]:

from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore


project_id = PROJECT_ID
location = LOCATION = "us" # Format is "us" or "eu"
PROCESSOR_DISPLAY_NAME = 'form_scanner_test' # Must be unique per project, e.g.: 'My Processor'
PROCESSOR_TYPE = 'FORM_PARSER_PROCESSOR' # Use fetch_processor_types to get available processor types



def create_processor(
    project_id: str, location: str, processor_display_name: str, processor_type: str
) -> None:
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    processor = client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_=processor_type
        ),
    )

    # Print the processor information
    print(f"Processor Name: {processor.name}")
    print(f"Processor Display Name: {processor.display_name}")
    print(f"Processor Type: {processor.type_}")


In [7]:
processorList=list_processors(PROJECT_ID,LOCATION)

def createUniqueProcessor():
    if len(processorList)==0:
        PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
        return PROCESSOR, PROCESSOR_ID
    else:
        for processor in processorList:
            if PROCESSOR_DISPLAY_NAME==processor.display_name:
                PROCESSOR_ID=processor.name.split('/')[-1]
                PROCESSOR=processor
                return PROCESSOR, PROCESSOR_ID
            else:
                try:
                    PROCESSOR, PROCESSOR_ID = create_processor(PROJECT_ID, LOCATION,PROCESSOR_DISPLAY_NAME, PROCESSOR_TYPE)
                    return PROCESSOR, PROCESSOR_ID
                except:
                    continue
                

PROCESSOR, PROCESSOR_ID = createUniqueProcessor()

In [8]:
from typing import Optional, Sequence

from google.api_core.client_options import ClientOptions
from google.cloud import documentai

# TODO(developer): Uncomment these variables before running the sample.
project_id = PROJECT_ID
location = "us" # Format is "us" or "eu"
processor_id = PROCESSOR_ID # Create processor before running sample
processor_version = "rc" # Refer to https://cloud.google.com/document-ai/docs/manage-processor-versions for more information
file_path = "./exampleform.pdf"
mime_type = "application/pdf" # Refer to https://cloud.google.com/document-ai/docs/file-types for supported file types


def process_document_form(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    # Online processing request to Document AI
    document = process_document(
        project_id, location, processor_id, processor_version, file_path, mime_type
    )

    # Read the table and form fields output from the processor
    # The form processor also contains OCR data. For more information
    # on how to parse OCR data please see the OCR sample.

    text = document.text
    print(f"Full document text: {repr(text)}\n")
    print(f"There are {len(document.pages)} page(s) in this document.")

    # Read the form fields and tables output from the processor
    for page in document.pages:
        print(f"\n\n**** Page {page.page_number} ****")

        print(f"\nFound {len(page.tables)} table(s):")
        for table in page.tables:
            num_columns = len(table.header_rows[0].cells)
            num_rows = len(table.body_rows)
            print(f"Table with {num_columns} columns and {num_rows} rows:")

            # Print header rows
            print("Columns:")
            print_table_rows(table.header_rows, text)
            # Print body rows
            print("Table body data:")
            print_table_rows(table.body_rows, text)

        print(f"\nFound {len(page.form_fields)} form field(s):")
        for field in page.form_fields:
            name = layout_to_text(field.field_name, text)
            value = layout_to_text(field.field_value, text)
            print(f"    * {repr(name.strip())}: {repr(value.strip())}")

    # Supported in version `pretrained-form-parser-v2.0-2022-11-10` and later.
    # For more information: https://cloud.google.com/document-ai/docs/form-parser
    if document.entities:
        print(f"Found {len(document.entities)} generic entities:")
        for entity in document.entities:
            print_entity(entity)
            # Print Nested Entities
            for prop in entity.properties:
                print_entity(prop)

    return document


def print_table_rows(
    table_rows: Sequence[documentai.Document.Page.Table.TableRow], text: str
) -> None:
    for table_row in table_rows:
        row_text = ""
        for cell in table_row.cells:
            cell_text = layout_to_text(cell.layout, text)
            row_text += f"{repr(cell_text.strip())} | "
        print(row_text)




def print_entity(entity: documentai.Document.Entity) -> None:
    # Fields detected. For a full list of fields for each processor see
    # the processor documentation:
    # https://cloud.google.com/document-ai/docs/processors-list
    key = entity.type_

    # Some other value formats in addition to text are available
    # e.g. dates: `entity.normalized_value.date_value.year`
    text_value = entity.text_anchor.content or entity.mention_text
    confidence = entity.confidence
    normalized_value = entity.normalized_value.text
    print(f"    * {repr(key)}: {repr(text_value)} ({confidence:.1%} confident)")

    if normalized_value:
        print(f"    * Normalized Value: {repr(normalized_value)}")




def process_document(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
    process_options: Optional[documentai.ProcessOptions] = None,
) -> documentai.Document:
    # You must set the `api_endpoint` if you use a location other than "us".
    client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor version, e.g.:
    # `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    # You must create a processor before running this sample.
    name = client.processor_version_path(
        project_id, location, processor_id, processor_version
    )

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=documentai.RawDocument(content=image_content, mime_type=mime_type),
        # Only supported for Document OCR processor
        process_options=process_options,
    )

    result = client.process_document(request=request)

    # For a full list of `Document` object attributes, reference this page:
    # https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
    return result.document




def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    return "".join(
        text[int(segment.start_index) : int(segment.end_index)]
        for segment in layout.text_anchor.text_segments
    )


In [88]:
exampleDocument = process_document_form(PROJECT_ID,LOCATION,PROCESSOR_ID,processor_version,file_path,mime_type)

import json
from google.cloud.documentai_v1 import Document

json_string = Document.to_json(exampleDocument)
dict_obj = json.loads(json_string)
with open("document.json", mode='w') as my_file:
    json.dump(dict_obj, my_file)

Full document text: 'FakeDoc M.D.\nHEALTH INTAKE FORM\nPlease fill out the questionnaire carefully. The information you provide will be used to completeyour health profile and will be kept confidential.\nDate:\n9/14/19\nName:\nSally Walker\nDOB: 09/04/1986\nAddress: 24 Barney Lane\nCity: Towaco\nState: NJ Zip: 07082\nEmail: Sally, walker@cmail.com\n_Phone #: (906) 917-3486\nGender: F\nMarital Status:\nSingle Occupation: Software Engineer\nReferred By: None\nEmergency Contact: Eva Walker Emergency Contact Phone: (906)334-8926\nDescribe your medical concerns (symptoms, diagnoses, etc):\nRanny nose, mucas in thwat, weakness,aches, chills, tired\nAre you currently taking any medication? (If yes, please describe):\nVyvanse (25mg) daily for attention.\n'

There are 1 page(s) in this document.


**** Page 1 ****

Found 0 table(s):

Found 17 form field(s):
    * 'Are you currently taking any medication? (If yes, please describe):': 'Vyvanse (25mg) daily for attention.'
    * '_Phone #:': 'walk

In [83]:
import os
from google.cloud import storage
from PyPDF2 import PdfReader

input_files = []

storage_client = storage.Client()
bucket = storage_client.bucket(document_bucket_name)

def download_pdf_from_gcs(bucket_name, source_blob_name, destination_folder):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    fileName = source_blob_name.split('/')[-1]
    blob.download_to_filename(f'{download_folder}/{fileName}')


# Create the destination folder if it doesn't exist
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Download all PDF files from the bucket
pdf_found = False 

# for blob in storage_client.list_blobs(document_bucket_name,prefix='input/'):
#     print(blob.name)

for blob in storage_client.list_blobs(document_bucket_name, prefix='input/'):   
    print(blob.name)
    if blob.name.endswith(".pdf"):
        download_pdf_from_gcs(document_bucket_name, blob.name, download_folder)
        pdf_found = True
        
if not pdf_found:
    print("No PDFs found in the bucket")

input/
input/exampleform (1).pdf
input/exampleform.pdf


In [84]:
import os

def list_files_in_directory(directory_path):

    file_list = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_list.append(file_path)
    return file_list

directory_path = "./forms"

fileList = list_files_in_directory(directory_path)

print(fileList)

['./forms/exampleform.pdf', './forms/exampleform (1).pdf']


In [170]:
# pdf_file = Part.from_uri(
#     uri="gs://thai-form-scanner/exampleform.pdf",
#     mime_type="application/pdf",
# )
outputList = []

def generate(pdf):
    contents = [form_prompt, json_string, prompt, pdf]
    response = model.generate_content(contents, tools = [form_tool])
    
    outputList.append(response)
    
    print(pdf + " has been processed.")

In [175]:
for file in fileList:
    generate(file)

./forms/exampleform.pdf has been processed.
./forms/exampleform (1).pdf has been processed.


In [201]:
import csv

with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = ["Id", "Name", "DOB", "Phone Number"]
    writer.writerow(field)

In [213]:
import pandas as pd

df = pd.read_csv('data.csv')

idList = []
nameList = []
dobList = []
phoneList = []

id = 0

for response in outputList:
    content = response.candidates[0].content.parts[0].text
    contentJson = json.loads(content[7:-4])

    id += 1
    
    idList.append(id)
    nameList.append(contentJson['name'])
    dobList.append(contentJson['date of birth'])
    phoneList.append(contentJson['phone number'])

df['Id'] = idList    
df['Name'] = nameList
df['DOB'] = dobList
df['Phone Number'] = phoneList

df.head(5)


,Id,Name,DOB,Phone Number
0,1,Sally Walker,09/04/1986,(906) 917-3486


In [214]:
df.to_csv('final.csv')